#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [5]:
#collection.delete_many({})

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Nettoyer les données

In [5]:
df_ks.shape

(150000, 14)

In [6]:
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                 object
launched             object
pledged              object
state                object
backers              object
country              object
usd pledged          object
usd_pledged_real    float64
dtype: object

In [7]:
df_ks.drop(66141,0,inplace=True)

df_ks = df_ks.drop(['ID','currency','deadline','backers'], axis=1)

df_ks["launched"] = df_ks["launched"].str[:4] 

df_ks = df_ks.rename({"usd pledged":"usd_pledged"}, axis='columns')

In [8]:
df_ks['name'] = df_ks['name'].astype('string')
df_ks['category'] = df_ks['category'].astype('string')
df_ks['main_category'] = df_ks['main_category'].astype('string')
#df_ks['currency'] = df_ks['currency'].astype('string')
#df_ks['deadline'] = pd.to_datetime(df_ks['deadline'])
df_ks['goal'] = df_ks['goal'].astype('float64')
df_ks['launched'] = df_ks['launched'].astype('int64')
df_ks['pledged'] = df_ks['pledged'].astype('float64')
df_ks['state'] = df_ks['state'].astype('string')
#df_ks['backers'] = df_ks['backers'].astype('float64')
df_ks['country'] = df_ks['country'].astype('string')
df_ks['usd_pledged'] = df_ks['usd_pledged'].astype('float64')

df_ks.dtypes


name                 string
category             string
main_category        string
goal                float64
launched              int64
pledged             float64
state                string
country              string
usd_pledged         float64
usd_pledged_real    float64
dtype: object

In [9]:
df_ks = df_ks.dropna()

df_ks.head()

,name,category,main_category,goal,launched,pledged,state,country,usd_pledged,usd_pledged_real
0,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,2000.0,2011,1145.0,canceled,US,1145.000000,1145.000000
1,Ohceola jewelry,Fashion,Fashion,18000.0,2012,1851.0,failed,US,1851.000000,1851.000000
2,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,2000.0,2016,7534.0,successful,US,3796.000000,7534.000000
3,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,13000.0,2017,16298.0,successful,US,2670.000000,16298.000000
4,Proven sales with custom motorcycle accessories,Sculpture,Art,5000.0,2016,1.0,failed,CA,0.708148,0.738225


### Importer les données

In [10]:
dict_df = df_ks.to_dict(orient='records')



In [11]:
dict_df

[{'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
  'category': 'Rock',
  'main_category': 'Music',
  'goal': 2000.0,
  'launched': 2011,
  'pledged': 1145.0,
  'state': 'canceled',
  'country': 'US',
  'usd_pledged': 1145.0,
  'usd_pledged_real': 1145.0},
 {'name': 'Ohceola jewelry',
  'category': 'Fashion',
  'main_category': 'Fashion',
  'goal': 18000.0,
  'launched': 2012,
  'pledged': 1851.0,
  'state': 'failed',
  'country': 'US',
  'usd_pledged': 1851.0,
  'usd_pledged_real': 1851.0},
 {'name': 'Sluff Off & Harald: Two latest EGGs are Classics "old & new"',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'goal': 2000.0,
  'launched': 2016,
  'pledged': 7534.0,
  'state': 'successful',
  'country': 'US',
  'usd_pledged': 3796.0,
  'usd_pledged_real': 7534.0},
 {'name': 'SketchPlanner: Create and Plan- all in one beautiful book!',
  'category': 'Art Books',
  'main_category': 'Publishing',
  'goal': 13000.0,
  'launched': 2017,
  'pledged': 16298.0,
  'st

In [12]:
collection.insert_many(dict_df)

## Question 1  

In [13]:
cur = collection.find().sort([("pledged",-1)]).limit(5)

list(cur)

[{'_id': ObjectId('5fca2048f4c9c8ca3e4ec1e8'),
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'goal': 50000.0,
  'launched': 2014,
  'pledged': 13285226.36,
  'state': 'successful',
  'country': 'US',
  'usd_pledged': 13285226.36,
  'usd_pledged_real': 13285226.36},
 {'_id': ObjectId('5fca2048f4c9c8ca3e4f1c12'),
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'category': 'Product Design',
  'main_category': 'Design',
  'goal': 1000000.0,
  'launched': 2016,
  'pledged': 12779843.49,
  'state': 'successful',
  'country': 'US',
  'usd_pledged': 12779843.49,
  'usd_pledged_real': 12779843.49},
 {'_id': ObjectId('5fca2048f4c9c8ca3e4ece00'),
  'name': 'Expect the Unexpected. digiFilmï¿½ Camera by YASHICA',
  'category': 'Product Design',
  'main_category': 'Design',
  'goal': 800000.0,
  'launched': 2017,
  'pledged': 10035296.0,
  'state': 'successful',
  'country': 'HK',
  'usd_pledged': 4788.642

## Question 2

In [14]:
cur = collection.count_documents({"state":"successful"})
cur

52998

## Question 3

In [15]:
cur = collection.aggregate([{"$group" : {"_id" : "$category", "nombre" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Sound', 'nombre': 255},
 {'_id': 'Farms', 'nombre': 482},
 {'_id': 'Country & Folk', 'nombre': 1790},
 {'_id': 'Science Fiction', 'nombre': 274},
 {'_id': 'Film & Video', 'nombre': 3657},
 {'_id': 'Drinks', 'nombre': 995},
 {'_id': 'Painting', 'nombre': 1288},
 {'_id': 'Nature', 'nombre': 219},
 {'_id': 'Residencies', 'nombre': 32},
 {'_id': 'Translations', 'nombre': 76},
 {'_id': 'Taxidermy', 'nombre': 7},
 {'_id': 'Playing Cards', 'nombre': 963},
 {'_id': 'Woodworking', 'nombre': 433},
 {'_id': 'Food', 'nombre': 4612},
 {'_id': 'Music Videos', 'nombre': 299},
 {'_id': 'Literary Spaces', 'nombre': 10},
 {'_id': 'Weaving', 'nombre': 38},
 {'_id': 'Conceptual Art', 'nombre': 393},
 {'_id': 'Pottery', 'nombre': 40},
 {'_id': 'Civic Design', 'nombre': 130},
 {'_id': 'Restaurants', 'nombre': 1088},
 {'_id': 'Graphic Novels', 'nombre': 702},
 {'_id': 'Audio', 'nombre': 164},
 {'_id': 'Fantasy', 'nombre': 132},
 {'_id': 'Photography', 'nombre': 2239},
 {'_id': 'Performances', 'nomb

## Question 4

In [18]:
cur = collection.count_documents({"$and":[{"country":"FR"}, {"launched":{"$lt": 2016}}]})
cur

330

## Question 5

In [19]:
cur = collection.find({"$and":[{"country":"US"}, {"goal":{"$gte": 200000}}]})

cur_nb = cur.count()
print(cur_nb)

list(cur)

<ipython-input-19-aa6ff32004d4>:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  cur_nb = cur.count()


2170


[{'_id': ObjectId('5fca2048f4c9c8ca3e4d9a59'),
  'name': 'Far from Par is a movie about a man and a talking golf ball.',
  'category': 'Comedy',
  'main_category': 'Film & Video',
  'goal': 200000.0,
  'launched': 2014,
  'pledged': 10.0,
  'state': 'failed',
  'country': 'US',
  'usd_pledged': 10.0,
  'usd_pledged_real': 10.0},
 {'_id': ObjectId('5fca2048f4c9c8ca3e4d9a6c'),
  'name': 'A CALL TO ADVENTURE',
  'category': 'Film & Video',
  'main_category': 'Film & Video',
  'goal': 287000.0,
  'launched': 2012,
  'pledged': 1465.0,
  'state': 'failed',
  'country': 'US',
  'usd_pledged': 1465.0,
  'usd_pledged_real': 1465.0},
 {'_id': ObjectId('5fca2048f4c9c8ca3e4d9b1e'),
  'name': 'Storybricks, the storytelling online RPG',
  'category': 'Video Games',
  'main_category': 'Games',
  'goal': 250000.0,
  'launched': 2012,
  'pledged': 23680.54,
  'state': 'failed',
  'country': 'US',
  'usd_pledged': 23680.54,
  'usd_pledged_real': 23680.54},
 {'_id': ObjectId('5fca2048f4c9c8ca3e4d9b26'),

## Question 6 

In [20]:
collection.create_index([("name",  "text")])
collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'name_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('name', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [21]:
cur = collection.count_documents( { "$text": { "$search": "Sport" } } )
cur

316